In [ ]:
############################################
# 1. SETUP & IMPORTS
############################################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import (
    mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, roc_auc_score
)

############################################
# 2. LOAD & COMBINE DATA
############################################
base_dir     = Path().cwd()  # assumes you launch the notebook from repo root
matches_dir  = base_dir / 'Project Datasets' / 'Premier League'
weather_dir  = base_dir / 'Project Datasets' / 'Weather'

# load all match-season files
match_files = sorted(matches_dir.glob('*.csv'))
print("Found match files:", [f.name for f in match_files])
matches = pd.concat(
    (pd.read_csv(f, parse_dates=['date']) for f in match_files),
    ignore_index=True
)

# load all weather files in all subfolders
weather_files = sorted(weather_dir.glob('**/*.csv'))
print("Found weather files:", [f.relative_to(weather_dir) for f in weather_files])
weather = pd.concat(
    (pd.read_csv(f, parse_dates=['datetime']) for f in weather_files),
    ignore_index=True
)

############################################
# 3. MERGE & FEATURE ENGINEERING
############################################
# 3.1 Merge on nearest datetime
matches  = matches.sort_values('date')
weather  = weather.sort_values('datetime')
df       = pd.merge_asof(
    matches, weather,
    left_on='date', right_on='datetime',
    direction='nearest'
)

# 3.2 New features
df['total_goals']    = df['home_goals'] + df['away_goals']
df['rain_flag']      = (df['precipitation_mm'] > 0).astype(int)
df['temp_cat']       = pd.cut(
                          df['temperature_C'],
                          bins=[-10,5,15,25,40],
                          labels=['cold','cool','warm','hot']
                      )
df['kickoff_hour']   = df['date'].dt.hour
df['day_of_week']    = df['date'].dt.day_name()

# 3.3 Rolling 3‐match form goals
for side in ['home','away']:
    df[f'{side}_form_goals'] = (
        df.groupby(f'{side}_team')['total_goals']
          .rolling(window=3, min_periods=1)
          .mean()
          .reset_index(level=0, drop=True)
    )

############################################
# 4. EXPLORATORY DATA ANALYSIS (EDA)
############################################
plt.figure(figsize=(6,4))
sns.boxplot(x='rain_flag', y='total_goals', data=df)
plt.title('Total Goals: Rain vs No Rain')
plt.show()

plt.figure(figsize=(6,4))
sns.barplot(
    x='temp_cat', y='total_goals', data=df,
    order=['cold','cool','warm','hot']
)
plt.title('Avg Goals by Temperature Category')
plt.show()

corr = df[['total_goals','rain_flag','temperature_C','humidity','attendance']].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, annot=True, fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

############################################
# 5. HYPOTHESIS TESTING
############################################
rain_goals = df[df['rain_flag']==1]['total_goals']
dry_goals  = df[df['rain_flag']==0]['total_goals']
tstat, pval = ttest_ind(rain_goals, dry_goals, equal_var=False)
print(f"T-test Rain vs No Rain → t = {tstat:.3f}, p = {pval:.3f}")

############################################
# 6. PREPARE FOR ML
############################################
features = [
    'rain_flag','temperature_C','humidity','attendance',
    'home_form_goals','away_form_goals'
]
X     = df[features].fillna(0)
y_reg = df['total_goals']
y_clf = (df['total_goals'] >= 3).astype(int)

X_train, X_test, y_train_reg, y_test_reg = train_test_split(
    X, y_reg, test_size=0.2, random_state=42
)
_, _, y_train_clf, y_test_clf = train_test_split(
    X, y_clf, test_size=0.2, random_state=42
)

############################################
# 7. REGRESSION MODELS
############################################
models_reg = {
    'LinearRegression'         : LinearRegression(),
    'RandomForestRegressor'    : RandomForestRegressor(n_estimators=100, random_state=42)
}
for name, model in models_reg.items():
    model.fit(X_train, y_train_reg)
    preds = model.predict(X_test)
    print(f"{name} → RMSE: {mean_squared_error(y_test_reg,preds, squared=False):.3f}, "
          f"R2: {r2_score(y_test_reg,preds):.3f}")

############################################
# 8. CLASSIFICATION MODELS
############################################
models_clf = {
    'LogisticRegression'       : LogisticRegression(max_iter=1000),
    'RandomForestClassifier'   : RandomForestClassifier(n_estimators=100, random_state=42)
}
for name, model in models_clf.items():
    model.fit(X_train, y_train_clf)
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:,1]
    print(f"{name} → Acc: {accuracy_score(y_test_clf,preds):.3f}, "
          f"Prec: {precision_score(y_test_clf,preds):.3f}, "
          f"Rec: {recall_score(y_test_clf,preds):.3f}, "
          f"AUC: {roc_auc_score(y_test_clf,probs):.3f}")

############################################
# 9. FEATURE IMPORTANCES (RF CLASSIFIER)
############################################
rf          = models_clf['RandomForestClassifier']
importances = pd.Series(rf.feature_importances_, index=features).sort_values(ascending=False)
plt.figure(figsize=(6,4))
importances.plot.bar()
plt.title('Feature Importances – RF Classifier')
plt.show()

############################################
# 10. CONCLUSIONS
############################################
best_reg = max(models_reg,
               key=lambda n: r2_score(y_test_reg, models_reg[n].predict(X_test)))
best_clf = max(models_clf,
               key=lambda n: roc_auc_score(y_test_clf,
                                           models_clf[n].predict_proba(X_test)[:,1]))
print("\n=== KEY FINDINGS ===")
print(f"- Rain vs No Rain significant (p<0.05)? → {'Yes' if pval<0.05 else 'No'}")
print(f"- Best regression model: {best_reg}")
print(f"- Best classification model: {best_clf}")
